In [2]:
#importing Libraries

from tensorflow.keras.layers import Dense,Flatten,Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.applications.xception import Xception,preprocess_input
from glob import glob
import numpy as np
import matplotlib.pyplot as plt 


In [3]:
# Data Collection 

Image_Size = [224,224]

train_path = r"/content/drive/MyDrive/Dataset/train"

test_path = r"/content/drive/MyDrive/Dataset/test"



In [4]:
# Image Preprocesssing

train_datagen=ImageDataGenerator(rescale=1./255,
                                 shear_range=0.2,
                                 zoom_range=0.2,
                                 horizontal_flip=True)

test_datagen=ImageDataGenerator(rescale=1./255)




In [5]:
training_set=train_datagen.flow_from_directory(train_path,
                                               target_size=(224,224),
                                               batch_size=32,
                                               class_mode='categorical')


test_set=test_datagen.flow_from_directory(test_path,
                                               target_size=(224,224),
                                               batch_size=32,
                                               class_mode='categorical')




                                                            

Found 911 images belonging to 3 classes.
Found 306 images belonging to 3 classes.


In [6]:
# Model Building

xception = Xception(input_shape=Image_Size + [3],weights='imagenet',include_top=False)




83683744/83683744 [==============================] - 1s 0us/step


In [7]:
for layer in xception.layers:
    layer.trainable = False

In [8]:
x = Flatten()(xception.output)
prediction = Dense(3,activation='softmax')(x)

In [9]:
model = Model(inputs=xception.input,outputs=prediction)

In [10]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 111, 111, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, 111, 111, 32  128        ['block1_conv1[0][0]']           
 ation)                         )                                                             

In [11]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics = ['accuracy'],
)

In [12]:
r = model.fit(
    training_set,
    validation_data = test_set,
    epochs=20,
    steps_per_epoch = len(training_set)//5,
    validation_steps = len(test_set)//5,

)

Epoch 1/20
5/5 [==============================] - 69s 12s/step - loss: 5.9031 - accuracy: 0.3636 - val_loss: 5.0837 - val_accuracy: 0.4062
Epoch 2/20
5/5 [==============================] - 48s 10s/step - loss: 2.8862 - accuracy: 0.6687 - val_loss: 4.5818 - val_accuracy: 0.6094
Epoch 3/20
5/5 [==============================] - 33s 8s/step - loss: 2.5118 - accuracy: 0.7133 - val_loss: 4.5135 - val_accuracy: 0.5938
Epoch 4/20
5/5 [==============================] - 26s 6s/step - loss: 3.1673 - accuracy: 0.6375 - val_loss: 1.9973 - val_accuracy: 0.7812
Epoch 5/20
5/5 [==============================] - 27s 6s/step - loss: 2.1779 - accuracy: 0.7273 - val_loss: 2.1456 - val_accuracy: 0.7500
Epoch 6/20
5/5 [==============================] - 16s 3s/step - loss: 1.4726 - accuracy: 0.8062 - val_loss: 1.9659 - val_accuracy: 0.6875
Epoch 7/20
5/5 [==============================] - 15s 3s/step - loss: 2.1029 - accuracy: 0.7500 - val_loss: 2.3366 - val_accuracy: 0.6875
Epoch 8/20
5/5 [================

In [13]:
model.save("mushroom.h5")

In [14]:
# testing the model

training_set.class_indices

{'Boletus': 0, 'Lactarius': 1, 'Russula': 2}

In [18]:
img = image.load_img(r"/content/drive/MyDrive/NM Project (AI)/Dataset/test/Lactarius/0010_vF92MBYp4JA.jpg",target_size=(224,224))

In [20]:
x = image.img_to_array(img)

In [21]:
x.shape

(224, 224, 3)

In [22]:
import numpy as np
x = np.expand_dims(x,axis=0)
img_data = preprocess_input(x)
img_data.shape

(1, 224, 224, 3)

In [23]:
output = np.argmax(model.predict(img_data),axis=1)

1/1 [==============================] - 2s 2s/step


In [24]:
output

array([1])